# CH 19

## TOC<a id='toc'></a>
* [Ch19 Notes](#ch19_notes)

### CH19 Notes <a id='ch19_notes'></a>
[toc](#toc)
### Dynamic Attributes and Properties

* Data attributes and methods are collectively known as **attributes**
    - a method is just a callable attribute
* we can also create **properties**, which replaces a public data attriburte with **accessor methods**
    - agress with the *Uniform access principle*: all services offered by a module should be available through a uniform notation, which does not betray whether they are implemented through storage or computation

* nice trick: can use two context managers at the same time (three? more?) since Python 2.7 and 3.1
` with urlopen(URL) as remote, open(JSON, 'wb') as local:`
* another nice trick: FrozenJSON class dealing with Keyword Attributes [use `keyword.iskeyword()`]
``` 
def __init__(self, mapping):
    self.__data = []
    for key, value in mapping.items():
        if keyword.iskeyword(key):
            key += '_'
        self.__data[key] = value
```
    - something similar can be done to check if key is a valid identifier by using string class `s.isidentifier()` method
    

## Flexible Object Creation with __new__
* we often refer to `__init__`  as the contstructor method, but the special method that actually constructs an instance is the `__new__` method
    - its a class method (but doesnt need the @classmethod decorator)
        - first arg is cls, remaining args are the ones init gets (except self)
    - it must return an instance
    - that instance will in turn get passed to the first argument of `__init__`
        * but not always, if `__new__` returns instance of a different  class, the init is not called.
* init is actually the initializer
* we rarely call the actual constructor `__new__` because the implementation inherited from object usually suffices
* usually `__new__(cls, arg)` just calls `super().__new__(cls)`
    - most often super() is `object`
    - results is of class cls, even though object new was used
    - object new implemented in C

## Restructuring the OSCON Feed with shelve

Another cool trick:  
* common shortcut to build an isntance with attributes created from key word arguments
```
class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
```

* *properties* are class attributes designed to manage instance attributes (that is they don't manage class attributes)
    - there are other solutions for this

* one of the differing facts about static vs class methods is that class methods are easier to customize in subclasses.

# Using a property for attribute validation
* implementing a propoerty allows us to use a getter and setter, but not change the interface of "accessing property direclty" that is via dot notation
* it is often a good idea to use the setter even in the initializing, to do the validation there too.
* Syntax

```
@property
def weight(self):
    return self._weight
    
@weight.setter
def weight(self, value):
    if value > 0:
        self._weight = value
    else:
        raise ValueError
```
* claim: The decorated getter has a .setter attribute, which is also a decorator. This ties the getter and setter together.
* properties are implemented as descriptor classes themselves - ch20 is about descriptors

# A Proper Look at Properties
* Though used as a decorator, the property builtin is actually a class
    - class and function are often interchangeable - because no new operator for object instantiation. So invoking a constructor is no different than invoking a factory function
    - both can be used as decorators as long as they return a new callable
* property constructor signature: `property(fget=None, fset=None, fdel=None, doc=None)`
* all arguments are optional, and if a function is not provided for one, the corresponding operation is not allowed by resulting object
    - decorator syntax became available after property type became avaialbe, so used to call it like `weight = property(get_weight, set_weight)` outside of any method def in the class. It is a public class attribute
* The presence of a property in a class affect how attributes and instances of that class can be found in a somewhat surprising way


In [1]:
class Class:
    data = "class data attr"
    @property
    def prop(self):
        return "prop value"

In [2]:
obj = Class()

In [3]:
obj.data

'class data attr'

In [5]:
vars(obj)

{}

In [6]:
vars(Class)

mappingproxy({'__module__': '__main__',
              'data': 'class data attr',
              'prop': <property at 0x218a570f630>,
              '__dict__': <attribute '__dict__' of 'Class' objects>,
              '__weakref__': <attribute '__weakref__' of 'Class' objects>,
              '__doc__': None})

In [7]:
obj.prop

'prop value'

In [8]:
Class.prop

In [9]:
obj.data = 'overriding data'

In [10]:
Class.data

'class data attr'

In [11]:
obj.data

'overriding data'

In [12]:
vars(obj)

{'data': 'overriding data'}

In [13]:
obj.prop = 'overriding prop'

AttributeError: can't set attribute

In [14]:
obj.__dict__['prop'] = 'overriding prop'

In [15]:
obj.prop

'prop value'

In [16]:
getattr(obj, 'prop')

'prop value'

In [17]:
hasattr(obj, 'prop')

True

In [18]:
vars(obj)

{'data': 'overriding data', 'prop': 'overriding prop'}

In [19]:
Class.prop

In [20]:
obj.prop

'prop value'

In [21]:
Class.prop = "overriding class prop"

In [22]:
Class.prop, obj.prop

('overriding class prop', 'overriding prop')

<hr>
<br>
The main point is that an expression like `obj.attr` does not seacrch for attr starting with obj, the search actually starts with `obj.__class__`. And only if there is no property will Python look in object.
<br>
<hr>

In [30]:
class Class():
    attr1 = 3
    def __init__(self, attr2):
        self.attr2 = 4
        
    def attr3(self):
        return 5
    
    @property
    def attr4(self):
        return 6

In [31]:
vars(Class)

mappingproxy({'__module__': '__main__',
              'attr1': 3,
              '__init__': <function __main__.Class.__init__(self, attr2)>,
              'attr3': <function __main__.Class.attr3(self)>,
              'attr4': <property at 0x218a735fc70>,
              '__dict__': <attribute '__dict__' of 'Class' objects>,
              '__weakref__': <attribute '__weakref__' of 'Class' objects>,
              '__doc__': None})

In [32]:
obj = Class(5)

In [33]:
vars(obj)

{'attr2': 4}

In [34]:
hasattr(obj, 'attr1'),  hasattr(obj, 'attr2'), hasattr(obj, 'attr3'),  hasattr(obj, 'attr4')

(True, True, True, True)

# Property Documentation
* every python unit of code can have a docstring
* the info is stored in the `__doc__` attribute
* when the property is deployed as a decorator, the docstring of getter method is used
    - deployin via constructor you can pass the doc arg
    

### This is pretty neat!

In [36]:
help(Class)

Help on class Class in module __main__:

class Class(builtins.object)
 |  Class(attr2)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, attr2)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  attr3(self)
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |  
 |  attr4
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  attr1 = 3



### He creates a property factory
* a function that a storage name, creates getter and setter closures with that storage name, then passes them as args to a property(), which is then returned by function.
* this is used to create class attributes - which are properties

* properties also have @property.deleter, even though we typically do not delete attributes

# Essential Attributes and Functions for Attribute Handling
* Special Attributes
    * `__class__`: reference to the object's class; 
        - `obj.__class__` is the same as `type(obj)`
        - python looks for special methods such as `__getarr__` only in the objects class and not in the instance itself (????)
    * `__dict__` - a mapping  that stores writable attributes of an object or class
        - if a class has `__slots__` attribute, then instance will not have a `__dict__`
    * `__slots__`: an attribute that may be defined in a class to limit attributes instances may have.
* Buit-in-Functions (for attrubte reading.. writing and introspection)
    * `dir([object])`: list *most* attributes of object.
        - intended for interactive purposes, so doesn't list all, just the "interesting" ones
        - if no object passed, dir list names in current scope
    * `getattr(object, name[, default])`:  get attribute, may fetch from class or super class.
        - if not found raises AttributeError, or returns default if given
    * `hasttr(object, name)`: returns True if attribute is in object, or can be fetched throug it (say by inheritance)
    * `setattr(object, name, value)`:  assigns value to named attribute if object allows it
    * `vars([object])`: returns `__dict__` of obj; 
        - cant deal with instances with slots
        - without object, it does the same thing as `locals()` (ie returns dict representing local scope)
* Special methods (attribute acces via dot, or using built-in funcs like getattr, hasattr and setatttr, trigger these special methods - reading and writing directly from dict does not)
    * "special methods are not shadowed by instance attributes of the same name" - so define them in class, not in instance `__dict__`
    * `__delattr__`: called when `del obj.attr`
    * `__dir__`: called when `dir(obj)`
    * `__getattribute__`: always called when there is an attempt to retrieve a named attribute, except when attribute sought is a special attribute od method.
        - implementation of `__getattribute__` should use `super().__getattribute__(obj, name)`
    * `__getattr__`: called only when an attrmpt to retrieve the named attrbiute fails, after Obj, class and ut superclasses are searched.
        - my be triggered by dot notation, getattr(), and hasattr()
        - only invoked after `__getattribute__` raises AttributeError
    * `__setattr__(self, name, value)___`: always called  when there is an attempt to set the named attribute. triggered by dot notation and setattr() func.

In [1]:
def myfunc(x):
    return x

In [2]:
type(myfunc)

function

In [4]:
type(myfunc.__class__)

type

## Soapbox
* `__new__` is better than `new` - a variation on the Uniform Access Principle (UAP)
    - in python syntax  `my_obj = foo()`  you dont know if foo is a func or a class, or any other callable
    - other languages have the `new` operator that distinguish these!
    - what this means is that replacing cosntructors with factories (which is much more flexible) or vice-versa, is quite easy in pyton, but not so in languages with `new`
    - and the existence of `__new__` can also make a class behave more like a factory function, because you can use it to return existing instance, or even instances of other classes